# Demonstração PyCaret

## Análise supervisionada com PyCaret no desafio do LABDATA da Fundação Instituto de Administração (FIA)

Realizaremos uma demonstração de aprendizagem supervisionada utilizando [PyCaret](https://pycaret.org/), uma biblioteca open source criada pelo cientista de dados e consultor da PwC Moez Ali. A biblioteca foi criada para facilitar a utilização de algoritmos de machine learning, podendo ser classificada como uma ferramenta da AutoML.

Para essa demonstração utilizaremos o [desafio no Kaggle](https://www.kaggle.com/c/labdata-churn-challenge-2020/data) proposto pela Fundação Instituto de Administração (FIA) para realizar a predição de quais clientes poderiam deixar uma certa companhia telefônica (churn). O colega Mário Monnerat trouxe o desafio para o treinamento de Big Data da Sufis, realizado em outubro e novembro de 2020, o que se mostrou uma ferramenta essencial para consolidar os conceitos de análise supervisionada.


Além da PyCaret utilizaremos as bibliotecas pandas, numpy

## Base de Churn

A base utilizada possui informações de cada cliente de uma companhia telefônica. Nosso intuito é prever se o cliente permanecerá contratando os serviços da empresa ou se vai sair (churn). Para tal, nos é fornecida a base de treino, que possui as informações de churn e a base de testes, que não possui e será utilizada para a previsão final.

Como sabemos, devemos utilizar somente a base de treino para tomar decisões com relação a qual o melhor modelo. Desse modo, devemos ter atenção e nos valer de ferramentas como validação cruzada (cross validation) e evitar o vazamento de dados do teste para o treino. O PyCaret utiliza validação cruzada sempre que possível.

## Pergunta a ser respondida

Conforme prega o CRISP-DM, antes de iniciarmos qualquer modelo de machine learning, devemos nos questionar sobre qual problema queremos solucionar. Nos desafios do Kaggle isso resta facilitado, tendo em vista que já temos o problema para ser solucionado, mas o mesmo não ocorre na vida real. Sugere-se seguir os passos do CRISP-DM, com bastante atenção para a primeira fase, para que o trabalho tenha um norte bem definido. 

In [ ]:
from IPython.display import Image
Image(filename='./img/crisp.png') 

In [ ]:
# instalar o PyCaret
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
from pycaret.classification import *


In [ ]:
# Carregar dados de treinamento e de teste
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
display(train_df.head())
display(test_df.head())

In [ ]:
# Setar id como indice dos dataframes
train_df.set_index('id', inplace=True)
test_df.set_index('id', inplace=True)

In [ ]:
display(train_df.head())
display(test_df.head())

In [ ]:
# Verificando os tipos
train_df.dtypes

In [ ]:
# Verificando missing data
train_df.isnull().sum()

In [ ]:
# TotalChanges não é numerico, transformar
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')



In [ ]:
# Feature engineering
train_df.loc[train_df['TotalCharges'].isnull(), 'TotalCharges'] = train_df.loc[train_df['TotalCharges'].isnull(), 'MonthlyCharges'] * train_df.loc[train_df['TotalCharges'].isnull(), 'tenure']
test_df.loc[test_df['TotalCharges'].isnull(), 'TotalCharges'] = test_df.loc[test_df['TotalCharges'].isnull(), 'MonthlyCharges'] * test_df.loc[test_df['TotalCharges'].isnull(), 'tenure']

In [ ]:
# Feature engineering + tratamento missing data
train_df['tenure'] = train_df['TotalCharges'] / train_df['MonthlyCharges']
test_df['tenure'] = test_df['TotalCharges'] /  test_df['MonthlyCharges']

In [ ]:
train_df.loc[train_df['Dependents'].isnull(), 'Dependents'] = train_df.loc[train_df['Dependents'].isnull(), 'MultipleLines']
test_df.loc[test_df['Dependents'].isnull(), 'Dependents'] = test_df.loc[test_df['Dependents'].isnull(), 'MultipleLines']

In [ ]:
train_df.loc[train_df['Dependents'] == 'No phone service', 'Dependents'] = 'No'
test_df.loc[test_df['Dependents'] == 'No phone service', 'Dependents'] = 'No'

In [ ]:
display(train_df.head())
display(test_df.head())

## A função setup permite muitas customizações para treinamento dos modelos

A função setup permite customizar (dentre outros):
- tratamento de missing data simples e com algoritmos
- over e undersampling
- principal component analysis (PCA)
- variaveis polinomiais
- razão entre variáveis
- % de treino e validação
- tratamento de colinearidade
- normalização

In [ ]:
# Função principal! Todas as configurações para treinamento dos modelos se dá no setup.

clf1 = setup(train_df, target = 'Churn', session_id=123, experiment_name='teste_1', feature_ratio=True, )

## Função que compara vários modelos de uma só vez, com várias métricas

One Function to rule them all, One Function to find them, One Function to bring them all, and in the darkness of jupyter notebook's theme bind them

In [ ]:
Image(filename='./img/ring.jpg') 

In [ ]:
# Testa 15 modelos de uma vez
best_model = compare_models(exclude=['catboost'])

In [ ]:
# Criando os melhores modelos de acordo com acurácia
modelo_lr = create_model('lr')
modelo_gbc = create_model('gbc')
modelo_ada = create_model('ada')
modelo_lightgbm = create_model('lightgbm')
#modelo_catboost = create_model('catboost')
modelo_ridge = create_model('ridge')
modelo_lda = create_model('lda')

In [ ]:
# Tunar os hiperparametros (pode demorar mais, pode tentar só com o create_model)
tuned_lr = tune_model(modelo_lr)
tuned_gbc = tune_model(modelo_gbc)
tuned_ada = tune_model(modelo_ada)
tuned_lightgbm = tune_model(modelo_lightgbm)
#tuned_catboost = tune_model(modelo_catboost)
tuned_ridge = tune_model(modelo_ridge)
tuned_lda = tune_model(modelo_lda)

In [ ]:
# Finaliza os modelos, ou seja, com os melhores hiperparametros, treinar na base de treino completa
f_lr = finalize_model(tuned_lr)
f_gbc = finalize_model(tuned_gbc)
f_ada = finalize_model(tuned_ada)
f_lightgbm = finalize_model(tuned_lightgbm)
#f_catboost = finalize_model(tuned_catboost)
f_ridge = finalize_model(tuned_ridge)
f_lda = finalize_model(tuned_lda)

## Criação de ensembles

Combinação de modelos pode ser útil para melhoria e generalização do modelo. O PyCaret permite ensemble com blending e stacking. Blending é uma espécie de votação e stacking utiliza as predições de um modelo para servir de variáveis explanatórias para o próximo. Uma grande vantagem de ensembles é que pode tornar o modelo mais robusto, com predições de diferentes tipos de algoritmos. Sugere-se utilizar algoritmos com abordagens bem diferentes (ex: redes neurais, algoritmos de árvore e algoritmos lineares).

No exemplo vamos utilizar um blending com votação "hard", ou seja, utiliza cada predição como 0 ou 1 na votação. O método "soft" utiliza as probabilidades de saída de cada modelo, o que pode melhorar o tuning do modelo final.


In [ ]:
# Votacao com os 6 modelos
blender_hard = blend_models(estimator_list = [f_lightgbm,
                                                f_ada,
                                                f_lr,
                                                f_gbc,
                                              f_ridge,
                                              f_lda,
                                              # f_catboost
                                             #   tuned_gbc,
                                             #   tuned_catboost
                                             ], method = 'hard')

In [ ]:
# mostrando a matriz de confusao
plot_model(blender_hard, plot = 'confusion_matrix')

In [ ]:
# finaliza o modelo: treina com a base de treino inteira
f_blender_hard = finalize_model(blender_hard)

In [ ]:
# realiza as predições (duas colunas: score e label)
predictions = predict_model(f_blender_hard, data = test_df)
predictions

In [ ]:
# Busca os labels (0 e 1). 
# Pode-se fazer alguma analise do melhor ponto de corte do Score, pois o label pega 0,5 pra cima como 1
pred_values = predictions['Label']
pred_values

In [ ]:
ids = test_df.index
output = pd.DataFrame({'id': ids, 'Churn': pred_values})
output.to_csv("submission.csv", index=False)